In [68]:
#import pandas
import pandas as pd
#importing numpy
import numpy as np
#importing tensorflow
import tensorflow as tf
#importing keras from tensorflow
from tensorflow import keras
# importing Sequential from keras
from tensorflow.keras.models import Sequential
#importing Dense and Conv2D layers from keras
from tensorflow.keras.layers import Dense,Conv2D

In [73]:
df = tf.keras.datasets.fashion_mnist
(train_df,train_labl),(test_df,test_labl) = df.load_data()

Normalizing values

In [74]:
train_df=train_df/255.0
#scaling test data
test_df = test_df/255.0

In [75]:
train_df = train_df.reshape(len(train_df),28,28,1)
test_df = test_df.reshape(len(test_df),28,28,1)

We are using categorical cross-entropy, so we need to convert our output to one-hot-encoded

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

In [70]:
def build_model(hp):
    model = keras.Sequential([keras.layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
#importing random search
from kerastuner import RandomSearch
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5)
# search best parameter
tuner.search(train_df,train_labl,epochs=3,validation_data=(train_df,train_labl))

Trial 3 Complete [00h 10m 17s]
val_accuracy: 0.8961166739463806

Best val_accuracy So Far: 0.8961166739463806
Total elapsed time: 00h 19m 41s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
48                |96                |conv_1_filter
5                 |3                 |conv_1_kernel
32                |48                |conv_2_filter
3                 |5                 |conv_2_kernel
32                |96                |dense_1_units
0.001             |0.01              |learning_rate

Epoch 1/3
1674/1875 [=========================>....] - ETA: 6s - loss: 0.4304 - accuracy: 0.8449